In [73]:
import os 
import folium
import json
import rasterio
from rasterio.warp import transform_bounds
import geopandas as gpd

In [78]:
def read_json(filename: str) -> dict:
    with open(filename) as input:
        field = json.load(input)
    return field
aoi = read_json("aoi/aoi.geojson")
ndvi_diff = rasterio.open("Result/NDVIDiff.tiff")
dem =  rasterio.open("Result/DEM.tiff")
inventory_val = gpd.read_file("Inventory/landslide_inventory.shp")


In [75]:
bounds = transform_bounds(ndvi_diff.crs,'EPSG:4326',*ndvi_diff.bounds)

In [90]:
m = folium.Map([42.75, 141.96], zoom_start=11)
folium.GeoJson(aoi).add_to(m)

tile = folium.TileLayer(
      tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
      attr = 'Esri',
      name = 'Esri Satellite',
      overlay = False,
      control = True
      ).add_to(m)


folium.raster_layers.ImageOverlay(
    image=ndvi_diff.read(1)>0.4,
    bounds=[[bounds[1]-0.0002,bounds[0]],[bounds[3],bounds[2]]],
#     origin="lower",
    colormap=lambda x: (1, 0, 0, x),
#     mercator_project=True
).add_to(m)

folium.GeoJson(
    inventory_val.to_crs(epsg=4326).to_json(),
    name="Vector Overlay",
    style_function=lambda feature: {
        'fillColor': '#blue',
        'color': 'green',
        'weight': 2,
        'fillOpacity': 0.5,
    }
).add_to(m)

folium.LayerControl().add_to(m)
m

In [70]:
[[bounds[0],bounds[1]],[bounds[2],bounds[3]]]

(141.7708476878404, 42.554214415271076, 142.13551105289486, 42.945729300196376)